# Labelling review

In [3]:
import os
import btrack
import napari
import glob
import zarr
import numpy as np

from macrohet import dataio, visualise
from tqdm.auto import tqdm

In [5]:
btrack.__version__

'0.6.1.dev129'

### Define functions

In [2]:
def track(objects, masks, config_fn, search_radius = 20):

    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure(config_fn)
        # set max search radius
        tracker.max_search_radius = search_radius
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # redefine features so that both channels are included in track measurements
        tracker.features = list(objects[0].properties.keys())
        # append the objects to be tracked
        tracker.append(objects)
        # set the tracking volume
        tracker.volume=((0, masks.shape[-2]*scale_factor), (0, masks.shape[-1]*scale_factor))
        # track them (in interactive mode)
        tracker.track(step_size=25)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # store the tracks
        tracks = tracker.tracks

    return tracks

In [3]:
def viewer_tools(viewer):
    
    @viewer.bind_key("z", overwrite=True)
    def true_track(viewer):
        """
        Marks a track as true based on the cursor position in the Napari viewer.
    
        Parameters
        ----------
        viewer : napari.viewer.Viewer
            The Napari viewer instance.
    
        """
        # Scale the coordinates for the tracks layer
        scaled_coords = [viewer.cursor.position[0]] + [coord / scale_factor for coord in viewer.cursor.position[1:]]
    
        # Use scaled coords to extract track ID under cursor
        cell_ID = viewer.layers['napari_tracks'].get_value(scaled_coords)
    
        if not cell_ID:
            print('cell ID not found')
        else:
            # Add track label to track_dict
            track_performance_dict[int(cell_ID)] = True
    
            with open(os.path.join(track_performance_dir, f'{row, column}_track_assessment.json'), "w") as file:
                json.dump(track_performance_dict, file)
    
            print(f"{cell_ID}:True")
    
        track = [track for track in tracks if track.ID == cell_ID][0]
        points = [[track.t[i], track.y[i] * scale_factor, track.x[i] * scale_factor] for i in range(len(track))]
        name = 'true tracks'
    
        try:
            # If the layer exists, add the points
            viewer.layers[name].add(points)
        except:
            # If the layer does not exist, create a new layer
            viewer.add_points(points,
                              size=33,
                              symbol='star',
                              face_color='transparent',
                              edge_color='white',
                              edge_width=0.1,
                              name=name,
                              opacity=1
                              # scale=napari_scale
                              )
    
    
    @viewer.bind_key("x", overwrite=True)
    def false_track(viewer):
        """
        Marks a track as false based on the cursor position in the Napari viewer.
    
        Parameters
        ----------
        viewer : napari.viewer.Viewer
            The Napari viewer instance.
    
        """
        # Scale the coordinates for the tracks layer
        scaled_coords = [viewer.cursor.position[0]] + [coord / scale_factor for coord in viewer.cursor.position[1:]]
    
        # Use scaled coords to extract track ID under cursor
        cell_ID = viewer.layers['napari_tracks'].get_value(scaled_coords)
    
        if not cell_ID:
            print('cell ID not found')
        else:
            # Add track label to track_dict
            track_performance_dict[int(cell_ID)] = False
    
            with open(os.path.join(track_performance_dir, f'{row, column}_track_assessment.json'), "w") as file:
                json.dump(track_performance_dict, file)
    
            print(f"{cell_ID}:False")
    
        track = [track for track in tracks if track.ID == cell_ID][0]
        points = [[track.t[i], track.y[i] * scale_factor, track.x[i] * scale_factor] for i in range(len(track))]
        name = 'false tracks'
    
        try:
            # If the layer exists, add the points
            viewer.layers[name].add(points)
        except:
            # If the layer does not exist, create a new layer
            viewer.add_points(points,
                              size=33,
                              symbol='x',
                              face_color='transparent',
                              edge_color='white',
                              edge_width=0.1,
                              name=name,
                              opacity=1
                              # scale=napari_scale
                              )
            
    @viewer.bind_key("b", overwrite=True)
    def record_true_with_time(viewer):
        """
        Records True along with the time dimension value based on the cursor position in the Napari viewer.
        This is so that I can crop certain tracks that are True from the point of time recorded onwards.
    
        Parameters
        ----------
        viewer : napari.viewer.Viewer
            The Napari viewer instance.
    
        """
        # Scale the coordinates for the tracks layer
        scaled_coords = [viewer.cursor.position[0]] + [coord / scale_factor for coord in viewer.cursor.position[1:]]
    
        # Use scaled coords to extract track ID under cursor
        cell_ID = viewer.layers['napari_tracks'].get_value(scaled_coords)
    
        if not cell_ID:
            print('cell ID not found')
        else:
            # Get the current time from the viewer
            current_time = viewer.dims.current_step[0]
    
            # Add (cell_ID, current_time, True) to track_performance_dict
            track_performance_dict[int(cell_ID)] = (current_time, True)
    
            with open(os.path.join(track_performance_dir, f'{row, column}_track_assessment.json'), "w") as file:
                json.dump(track_performance_dict, file)
    
            print(f"{cell_ID}:{current_time}:True")
    
        track = [track for track in tracks if track.ID == cell_ID][0]
        points = [[track.t[i], track.y[i] * scale_factor, track.x[i] * scale_factor] for i in range(len(track))]
        name = 'true tracks'
    
        try:
            # If the layer exists, add the points
            viewer.layers[name].add(points)
        except:
            # If the layer does not exist, create a new layer
            viewer.add_points(points,
                              size=33,
                              symbol='triangle_down',
                              face_color='transparent',
                              edge_color='white',
                              edge_width=0.1,
                              name=name,
                              opacity=1
                              # scale=napari_scale
                              )

In [4]:
# define tracking scale factor
scale_factor = 1/5.04

# define tracker config fn to use, using a prob_not_assign = 0.1
config_fn = '/home/dayn/analysis/models/btrack/particle_config_pnassign.json'

### Define scope of review

In [5]:
expts_to_review = ['ND0002','ND0003']
positions_to_review = 'all'

### Iteratively load images and tracks to review

In [ ]:
for expt_ID in expts_to_review:

    base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
    metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
    metadata = dataio.read_harmony_metadata(metadata_fn)  
    metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
    assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)
    
    for (row, column), info in tqdm(assay_layout.iterrows(), desc='Progress through positions', total=len(assay_layout), leave = False):
    
        acq_ID = (row, column)
    
        # process images using zarr
        image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
        zarr_store = zarr.open(image_dir, mode='r')
        zarr_images = zarr_store.images
        
        # create a max projection
        # %time images = np.max(zarr_images, axis = 2)
        # for times sake only load z0
        images_z0 = zarr_images[:,:,0,...]
        # load objects and segmentation 
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_mask_backup.h5'),
                                                   'r', 
                                                   obj_type='obj_type_1'
                                                   ) as reader:
                            masks = reader.segmentation

        if os.path.exists(f'{expt_ID}.{row}.{column}_filtered_napari_tracks.npy'):
            print('Loading tracks')
            napari_tracks = np.load(f'{expt_ID}.{row}.{column}_filtered_napari_tracks.npy')
            
        else:
            print('Tracking tracks')
            with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_objects_backup.h5'),
                                                       'r', 
                                                       obj_type='obj_type_1'
                                                       ) as reader:
                                objects = reader.objects
    
            # retrack as fuggin tracks didn't save out
            tracks = track(objects, masks, config_fn, search_radius = 20)
    
            # apply length filter for tracks
            filtered_tracks = [t for t in tracks if len(t) >= 70]
    
            # convert to napari format
            napari_tracks, _, _ = btrack.utils.tracks_to_napari(filtered_tracks, ndim = 2)
            np.save(f'{expt_ID}.{row}.{column}_filtered_napari_tracks.npy', napari_tracks)
            np.save(f'{expt_ID}.{row}.{column}_napari_tracks.npy', btrack.utils.tracks_to_napari(tracks, ndim = 2))

        # launch  napari viewer
        viewer = napari.Viewer(title = f'{expt_ID}, {row, column}')
        viewer.add_image(images_z0, channel_axis=1, contrast_limits=[[0,1000], [0, 2000]])
        viewer.add_labels(masks)
        viewer.add_tracks(napari_tracks, scale = (1/scale_factor, 1/scale_factor))
        viewer_tools(viewer)
        visualise.add_napari_grid_overlay(viewer)
        viewer.show(block=True)

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


Progress through positions:   0%|          | 0/42 [00:00<?, ?it/s]

[INFO][2024/03/27 11:56:10 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 1)_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 1)_cpv3_mask_backup.h5...
[INFO][2024/03/27 11:56:31 AM] Loading segmentation (150, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (150, 6048, 6048)
[INFO][2024/03/27 11:56:31 AM] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 1)_cpv3_mask_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 1)_cpv3_mask_backup.h5


...

Loading tracks


[INFO][2024/03/27 01:44:29 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 5)_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 5)_cpv3_mask_backup.h5...
[INFO][2024/03/27 01:45:11 pm] Loading segmentation (150, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (150, 6048, 6048)
[INFO][2024/03/27 01:45:11 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 5)_cpv3_mask_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 5)_cpv3_mask_backup.h5


Loading tracks


[INFO][2024/03/27 04:36:25 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5...
[INFO][2024/03/27 04:37:16 pm] Loading segmentation (150, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (150, 6048, 6048)
[INFO][2024/03/27 04:37:16 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5


Loading tracks


In [ ]:
break

# Track all pos and save to dict 

In [ ]:
# track_dict = {}

for expt_ID in expts_to_review:

    base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
    metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
    metadata = dataio.read_harmony_metadata(metadata_fn)  
    metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
    assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)

    # track_dict[expt_ID] = {}
    
    for (row, column), info in tqdm(assay_layout.iterrows(), desc='Progress through positions', total=len(assay_layout), leave = False):

        if os.path.exists(f'{expt_ID}.{row}.{column}_filtered_napari_tracks.npy'):
            continue
        acq_ID = (row, column)
    
        # process images using zarr
        image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
        zarr_store = zarr.open(image_dir, mode='r')
        zarr_images = zarr_store.images
        
        # create a max projection
        # %time images = np.max(zarr_images, axis = 2)
        # for times sake only load z0
        images_z0 = zarr_images[:,:,0,...]

        # load objects and segmentation 
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_mask_backup.h5'),
                                                   'r', 
                                                   obj_type='obj_type_1'
                                                   ) as reader:
                            masks = reader.segmentation
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_objects_backup.h5'),
                                                   'r', 
                                                   obj_type='obj_type_1'
                                                   ) as reader:
                            objects = reader.objects

        # retrack as fuggin tracks didn't save out
        tracks = track(objects, masks, config_fn, search_radius = 20)
        napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)
        np.save(f'{expt_ID}.{row}.{column}_napari_tracks.npy', napari_tracks)

        
        # # apply length filter for tracks
        filtered_tracks = [t for t in tracks if len(t) >= 70]

        # # convert to napari format
        napari_tracks, _, _ = btrack.utils.tracks_to_napari(filtered_tracks, ndim = 2)
        np.save(f'{expt_ID}.{row}.{column}_filtered_napari_tracks.npy', napari_tracks)

        # # launch  napari viewer
        # viewer = napari.Viewer(title = f'{expt_ID}, {row, column}')
        # viewer.add_image(images_z0, channel_axis=1, contrast_limits=[[0,1000], [0, 2000]])
        # viewer.add_labels(masks)
        # viewer.add_tracks(napari_tracks, scale = (1/scale_factor, 1/scale_factor))
        # viewer_tools(viewer)
        # visualise.add_napari_grid_overlay(viewer)
        # viewer.show(block=True)
        # track_dict[]

In [ ]:
import os
import numpy as np
import tqdm
import zarr
import btrack
from glob import glob

# Assuming dataio, track, config_fn, viewer_tools, visualise, scale_factor are defined elsewhere

for expt_ID in expts_to_review:

    base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
    metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
    metadata = dataio.read_harmony_metadata(metadata_fn)  
    metadata_path = glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
    assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True)

    for (row, column), info in tqdm.tqdm(assay_layout.iterrows(), desc='Progress through positions', total=len(assay_layout), leave=False):
        try:
            if os.path.exists(f'{expt_ID}.{row}.{column}_filtered_napari_tracks.npy'):
                continue
            acq_ID = (row, column)
        
            
            # Process images using zarr
            image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
            zarr_store = zarr.open(image_dir, mode='r')
            zarr_images = zarr_store.images  # This is where the AttributeError might occur
    
            # For times sake only load z0
            images_z0 = zarr_images[:,:,0,...]
    
            
            # Continue with processing if no error occurred
            # (Your existing code for processing goes here, starting with loading objects and segmentation)
             # load objects and segmentation 
            with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_mask_backup.h5'),
                                                       'r', 
                                                       obj_type='obj_type_1'
                                                       ) as reader:
                                masks = reader.segmentation
            with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_objects_backup.h5'),
                                                       'r', 
                                                       obj_type='obj_type_1'
                                                       ) as reader:
                                objects = reader.objects
    
            # retrack as fuggin tracks didn't save out
            tracks = track(objects, masks, config_fn, search_radius = 20)
            napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)
            np.save(f'{expt_ID}.{row}.{column}_napari_tracks.npy', napari_tracks)
    
            
            # # apply length filter for tracks
            filtered_tracks = [t for t in tracks if len(t) >= 70]
    
            # # convert to napari format
            napari_tracks, _, _ = btrack.utils.tracks_to_napari(filtered_tracks, ndim = 2)
            np.save(f'{expt_ID}.{row}.{column}_filtered_napari_tracks.npy', napari_tracks)

        except AttributeError as e:
                print(f"Error accessing zarr_images for {expt_ID}, {row}, {column}: {e}")
                continue  # Skip to the next iteration


In [ ]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)


In [ ]:
napari_tracks.shape

### Fixing track IO error

In [ ]:
base_dir = f'/mnt/SYNO/macrohet_syno/ND0003/'

In [ ]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/(6, 12)_cpv3_mask_backup.h5'),
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                    # tracks_loaded = reader.tracks
                    masks_loaded = reader.segmentation
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/(6, 12)_cpv3_objects_backup.h5'),
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                    objects_loaded = reader.objects

In [ ]:
masks_loaded.shape

In [ ]:
objects_loaded

In [ ]:
objects = objects_loaded
masks = masks_loaded

In [ ]:
tracks = track(objects, masks, config_fn, search_radius = 20)

In [ ]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)

In [ ]:
with btrack.io.HDF5FileHandler('test.h5', 
                               'w', 
                               obj_type='obj_type_1'
                               ) as handler:
                    handler.write_tracks(tracks)

In [ ]:
len(tracks)

In [ ]:
dummies = [obj for obj in objects if obj.dummy]

In [ ]:
dummies

In [ ]:
len(objects)

In [ ]:
len([obj for obj in objects if not obj.dummy])

In [ ]:
tracks[0].dummy

In [ ]:
with btrack.io.HDF5FileHandler('test.h5', 
                               'r', 
                               obj_type='obj_type_1'
                               ) as reader:
                    loaded_tracks = reader.tracks

In [ ]:
loaded_tracks

In [ ]:
viewer = napari.Viewer(title = f'{expt_ID}, {row, column}')

viewer.add_image(images_z0, channel_axis=1, contrast_limits=[[0,1000], [0, 2000]])

viewer.add_labels(masks_loaded)

viewer.add_tracks(napari_tracks, scale = (1/scale_factor, 1/scale_factor))